### ⚠ IMPORTANT ⚠

Please ensure your Colab runtime is set to the following:

A100 GPU

Evaluation and instruction-tuning a LLM is a resource intensive process - please make sure you're using the appropriate instance.

# Model Evaluation: A Primer

Now that we've spent some time creating models with:

- Unsupervised pre-training -> nanoGPT -> Shapereare
- Supervised fine-tuning -> gpt2 -> b-mc2/sql-create-context -> Understand and write sql query
- Some instruction-tuning -> Mistral.v01 -> mosaicml/instruct-v3 -> Thought how to respond to the instruction

We're ready to begin to think about how we can evaluate these models.

## Baseline Evaluation

In order to properly understand how our model is improving - we need to first start with a baseline evaluation of our model's performance.

Let's start with Mistral AI's `Mistral-7B` model.

We're going to load and compare everything in 4-bit quantization today in order to ensure we can fit the model on our Google Colab instance.

Let's start by setting up and loading our model to prepare it for evaluation.

### Load Mistral AI's Mistral-7B in 4-bit Quantization

Let's grab our dependencies, and load our model!

In [ ]:
!pip install -qU bitsandbytes datasets accelerate loralib peft transformers trl

In [ ]:
import torch
import torch.nn as nn
import bitsandbytes as bnb
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM, BitsAndBytesConfig

Conforming to previous notebooks - let's set up our quantization config for our model.

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16,
)

Now we have our quantization settings confirmed - let's load up our model!

In [ ]:
from huggingface_hub import login
login(new_session=False)

In [ ]:
%%time
model_id = "mistralai/Mistral-7B-v0.1"

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map='auto',
)

tokenizer = AutoTokenizer.from_pretrained(model_id)

In [ ]:
#With Quantization
#!nvidia-smi

In [ ]:
#Without  Quantization
!nvidia-smi

In [ ]:
#Without Quantization GPU memory
#29073/1024


In [ ]:
#With Quantization GPU memory
#5441/1024

In [ ]:
# With Quantization
# CPU times: user 37.1 s, sys: 29.1 s, total: 1min 6s
# Wall time: 1min 2s

####❓ Question:

Taking a look at the [model card](https://huggingface.co/mistralai/Mistral-7B-v0.1) (and the linked resources on the card) is this an instruct-tuned model or not?

### Collect and Load the Eleuther AI Evaluation Harness

Now that we have our baseline model loaded - we need to evaluate it.

For that, we'll use a tool called [Eleuther AI's LM evaluation harness](https://github.com/EleutherAI/lm-evaluation-harness). This is a specialized tool for running benchmarks on various language tasks.

Let's start by grabbing and installing it!

Why Eleuther AI's Evaluation Harness? Well - it's what powers the [Open LLM Leaderboard](https://huggingface.co/spaces/HuggingFaceH4/open_llm_leaderboard)!

In [ ]:
!git clone https://github.com/EleutherAI/lm-evaluation-harness
%cd lm-evaluation-harness
!pip install -e .

Now, we can cast our model to the desired format.

In [ ]:
import lm_eval
from lm_eval.models.huggingface import HFLM
eval_model = HFLM(model, batch_size=4)

We'll set up our tasks so we can leverage them at evaluation time!

Next, we can evaluate our base model!

>NOTE: This step will take ~30-40min. to run in full on the A100 - so ensure you set aside time to run it fully if you desire!

We're going to leverage two benchmarks today:

- [HellaSwag](https://rowanzellers.com/hellaswag/)
- [ARC Easy](https://leaderboard.allenai.org/arc_easy/submissions/get-started)
- A subset of the [MMLU benchmark](https://paperswithcode.com/dataset/mmlu), focusing only on the `machine_learning` task.

These are lightweight benchmarks used to "score" models against eachother on the OpenLM leaderboard.

We'll consider a simple average of their scores as the "overall" score of the baseline model.

You could easily extend the number of tasks considered if you wanted to more exactly emulate the Open LLM Leaderboard.

In [ ]:
results = lm_eval.simple_evaluate(
    model=eval_model,
    tasks=["hellaswag", "arc_easy"],
    num_fewshot=0,
    batch_size=16,
)

In [ ]:
import pandas as pd

pd.DataFrame(results["results"])

In [ ]:
fs_mmlu_results = lm_eval.simple_evaluate(
    model=eval_model,
    tasks=["mmlu_flan_n_shot_loglikelihood_machine_learning"],
    num_fewshot=5,
    batch_size=16,
)

In [ ]:
import pandas as pd

pd.DataFrame(fs_mmlu_results["results"])

### Zero-Shot MMLU

In [ ]:
zs_mmlu_results = lm_eval.simple_evaluate(
    model=eval_model,
    tasks=["mmlu_flan_n_shot_loglikelihood_machine_learning"],
    num_fewshot=0,
    batch_size=16,
)

In [ ]:
import pandas as pd

pd.DataFrame(zs_mmlu_results["results"])

### Chain of Thought

Now let's try a Chain of Thought example!

In [ ]:
cot_mmlu_results = lm_eval.simple_evaluate(
    model=eval_model,
    tasks=["mmlu_flan_cot_zeroshot_machine_learning"],
    num_fewshot=0,
    batch_size=16,
)

In [ ]:
import pandas as pd

pd.DataFrame(cot_mmlu_results["results"])